# vb-mols

In [1]:
import lmdb
import pickle
from tqdm.auto import tqdm 
import pandas as pd

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/vib-to-mol/datasets/vibench/mols/mols_train.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_train = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/vib-to-mol/datasets/vibench/mols/mols_test.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_test = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/vib-to-mol/datasets/vibench/mols/mols_eval.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_eval = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 18786/18786 [00:01<00:00, 17006.12it/s]


In [2]:
with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9.pkl', 'rb') as f:
    qm9_df = pickle.load(f)

with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/zinc15.pkl', 'rb') as f:
    zinc15_df = pickle.load(f)

v2m_df = pd.concat([qm9_df, zinc15_df])

In [15]:
merged_df = pd.merge(mols_test, v2m_df, on='filename', how='inner')
print((mols_test.shape, merged_df.shape))

((37570, 6), (37570, 13))


In [16]:
import numpy as np
from tqdm import trange

batch_size = len(merged_df)

merged_df['num_atoms'] = [len(item) for item in merged_df.z]
max_atom_num = 95

print(batch_size, max_atom_num)

# train:131489
# eval:18786
# test:

index = np.arange(131489+18786, 131489+18786+batch_size)
num_atoms = np.zeros(batch_size)
charges = np.zeros((batch_size, max_atom_num))
positions = np.zeros((batch_size, max_atom_num, 3))

for i in trange(len(merged_df)):
    tmp_num_atoms = merged_df.iloc[i]['num_atoms']
    tmp_z = merged_df.iloc[i]['z']
    tmp_pos = merged_df.iloc[i]['pos']
    
    num_atoms[i] = tmp_num_atoms
    charges[i, :len(tmp_z)] = tmp_z
    positions[i, :len(tmp_pos)] = tmp_pos


37570 95


100%|██████████| 37570/37570 [00:40<00:00, 937.09it/s] 


In [17]:
import torch
result = {'index':torch.as_tensor(index), 
'num_atoms':torch.as_tensor(num_atoms), 
'charges':torch.as_tensor(charges), 
'positions': torch.as_tensor(positions), 
# 'IR': torch.as_tensor(np.stack(merged_df.raman_x))}
# with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/mols_train.pickle', 'wb') as f:
#     pickle.dump(result, f)

'IR': torch.as_tensor(np.stack(merged_df.ir_x))}
with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/mols_ir_test.pickle', 'wb') as f:
    pickle.dump(result, f)


In [6]:
merged_df_train = pd.merge(mols_train, v2m_df, on='filename', how='inner')
merged_df_eval = pd.merge(mols_eval, v2m_df, on='filename', how='inner')
merged_df_test = pd.merge(mols_test, v2m_df, on='filename', how='inner')
total_df = pd.concat([merged_df_train, merged_df_eval, merged_df_test])

In [7]:
# total_df = total_df[['filename', 'norm_smiles_x', 'raman_x']]
total_df = total_df[['filename', 'norm_smiles_x', 'ir_x']]
total_df.columns = ['ID', 'SMILES', 'IR_Data']

In [8]:
total_df['ID'] = [f'{item}_{str(i)}' for i, item in enumerate(total_df['ID'])]

In [9]:
total_df.head()

,ID,SMILES,IR_Data
0,ZINC000053172821_0,COCC(C)Nc1cc(C)ccc1F,"[0.0025720204091958863, 0.0025826469982336724,..."
1,ZINC000095733849_1,N#CCOc1ccc(F)c(F)c1F,"[0.0003724409905910961, 0.00036209873506661103..."
2,dsgdb9nsd_073446_2,CC12C3CCC1(O)C2O3,"[0.17816624459728858, 0.11880080517191945, 0.0..."
3,dsgdb9nsd_030654_3,CCC(O)c1ccco1,"[0.15402493744068, 0.26502243698532457, 0.4266..."
4,dsgdb9nsd_075558_4,C#CC1OCCN=C1N,"[0.019886485482094728, 0.02966831809291165, 0...."


In [10]:
# with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/mols_raman.pkl', 'wb') as f:
#     pickle.dump(total_df, f)

with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/mols_ir.pkl', 'wb') as f:
    pickle.dump(total_df, f)

# qm9s

In [2]:
import lmdb
import pickle
from tqdm.auto import tqdm 
import pandas as pd

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_train.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_train = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_test.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_test = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

db = lmdb.open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_eval.lmdb', subdir=False, lock=False, map_size=int(1e11))
with db.begin() as txn:
    data = list(txn.cursor())

mols_eval = pd.DataFrame([pickle.loads(item[1]) for item in tqdm(data)])

/root/miniconda3/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
100%|██████████| 5842/5842 [00:01<00:00, 5825.61it/s]


In [3]:
mols_test.head()

,smiles,norm_smiles,kekule_smiles,raman,ir,pos,z,formula
0,C1CC2(CC3NC23)O1,C1CC2(CC3NC32)O1,C1CC2(CC3NC32)O1,"[758.639209524991, 504.0434624586983, 385.8349...","[0.24978338, 0.18424135, 0.13247555, 0.0973020...","[[tensor(0.0090), tensor(1.3747), tensor(-0.30...","[tensor(6), tensor(6), tensor(6), tensor(6), t...",C6H9NO
1,CC12NC1C1NC21C#N,CC12NC1C1NC12C#N,CC12NC1C1NC12C#N,"[961.0252794107125, 709.4743827830415, 587.435...","[0.18608528, 0.19516823, 0.21484478, 0.2595096...","[[tensor(-0.0635), tensor(1.5485), tensor(0.09...","[tensor(6), tensor(6), tensor(7), tensor(6), t...",C6H7N3
2,CC1(C)NC1C(O)CO,CC1(C)NC1C(O)CO,CC1(C)NC1C(O)CO,"[241.29470249080182, 258.15962526894316, 211.2...","[0.42916986, 0.45265028, 0.41397473, 0.3520797...","[[tensor(0.2145), tensor(1.5764), tensor(-0.28...","[tensor(6), tensor(6), tensor(6), tensor(7), t...",C6H13NO2
3,CC1(C)CC(O)(CO)C1,CC1(C)CC(O)(CO)C1,CC1(C)CC(O)(CO)C1,"[462.7068167225568, 725.8493974091627, 987.409...","[0.31098047, 0.35038358, 0.40292433, 0.4709874...","[[tensor(0.4838), tensor(1.2783), tensor(0.192...","[tensor(6), tensor(6), tensor(6), tensor(6), t...",C7H14O2
4,OC1=CN=C(NC=O)N1,O=CNc1ncc(O)[nH]1,O=CNC1=NC=C(O)N1,"[774.8998424911796, 559.9218000125314, 435.213...","[0.099352665, 0.09671722, 0.09769174, 0.101848...","[[tensor(0.2335), tensor(1.3689), tensor(0.253...","[tensor(8), tensor(6), tensor(6), tensor(7), t...",C4H5N3O2


In [13]:
import numpy as np
from tqdm import trange

batch_size = len(mols_test)

mols_test['num_atoms'] = [len(item) for item in mols_test.z]
max_atom_num = 35

print(batch_size, max_atom_num)

# train:110992
# eval:5842
# test:

index = np.arange(5842+110992, 5842+110992+batch_size)
num_atoms = np.zeros(batch_size)
charges = np.zeros((batch_size, max_atom_num))
positions = np.zeros((batch_size, max_atom_num, 3))

for i in trange(len(mols_test)):
    tmp_num_atoms = mols_test.iloc[i]['num_atoms']
    tmp_z = mols_test.iloc[i]['z']
    tmp_pos = mols_test.iloc[i]['pos']
    
    num_atoms[i] = tmp_num_atoms
    charges[i, :len(tmp_z)] = tmp_z
    positions[i, :len(tmp_pos)] = tmp_pos


12982 35


100%|██████████| 12982/12982 [00:03<00:00, 3830.71it/s]


In [5]:
mols_test.iloc[0]['z']

tensor([6, 6, 6, 6, 6, 7, 6, 8, 1, 1, 1, 1, 1, 1, 1, 1, 1])

In [15]:
import torch
result = {'index':torch.as_tensor(index), 
'num_atoms':torch.as_tensor(num_atoms), 
'charges':torch.as_tensor(charges), 
'positions': torch.as_tensor(positions), 
# 'IR': torch.as_tensor(np.stack(mols_test.raman))}
# with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_raman_test.pickle', 'wb') as f:
#     pickle.dump(result, f)

'IR': torch.as_tensor(np.stack(mols_test.ir))}
with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_ir_test.pickle', 'wb') as f:
    pickle.dump(result, f)


In [22]:
total_df = pd.concat([mols_train, mols_eval, mols_test])

In [23]:
total_df = total_df[['norm_smiles', 'raman']]
# total_df = total_df[['norm_smiles', 'ir']]
total_df.columns = ['SMILES', 'IR_Data']
total_df['ID'] = [f'{str(i)}_{str(i)}' for i, item in enumerate(total_df['SMILES'])]

In [24]:
with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_raman.pkl', 'wb') as f:
    pickle.dump(total_df, f)

# with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_ir.pkl', 'wb') as f:
#     pickle.dump(total_df, f)

In [4]:
import pickle
with open('/fs-computility/ai4chem/luxinyu.p/Spectra2Structure/data/qm9s_ir_test.pickle', 'rb') as f:
    data = pickle.load(f)

In [5]:
data

{'index': tensor([116834, 116835, 116836,  ..., 129813, 129814, 129815]),
 'num_atoms': tensor([17., 16., 22.,  ..., 21., 20., 15.], dtype=torch.float64),
 'charges': tensor([[6., 6., 6.,  ..., 0., 0., 0.],
         [6., 6., 7.,  ..., 0., 0., 0.],
         [6., 6., 6.,  ..., 0., 0., 0.],
         ...,
         [6., 6., 6.,  ..., 0., 0., 0.],
         [6., 6., 6.,  ..., 0., 0., 0.],
         [8., 6., 6.,  ..., 0., 0., 0.]], dtype=torch.float64),
 'positions': tensor([[[ 0.0090,  1.3747, -0.3026],
          [ 1.4517,  1.5511,  0.1969],
          [ 1.2905,  0.1332,  0.7617],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[-0.0635,  1.5485,  0.0926],
          [ 0.0666,  0.0623,  0.0436],
          [-0.8483, -0.8789,  0.7025],
          ...,
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000],
          [ 0.0000,  0.0000,  0.0000]],
 
         [[ 0.2145,  1.5764, -0.28